# Reshaping Dataframes

In [1]:
import pandas as pd

## Pivot
- In pandas, there are two methods `.pivot()` and `.pivot_table()` (RECOMMENDED)
- However, `.pivot()` unable to handle duplicate values in the index column, in this case, the index column is `cusid` which contains multiple rows of `cusid=1`, and `cusid=2`

In [22]:
df = pd.DataFrame({'cusid': [1,1,1,2,2,2],
                   'payment_method': ['DEBIT', 'DEBIT', 'CREDIT', 'CREDIT', 'CREDIT', 'DEBIT'],
                   'merchant': ['SHOPEE', 'GRAB', 'SHOPEE', 'SHOPEE', 'LAZADA', 'GSM'],
                   'total_txn': [1, 2, 3, 4, 5, 6],
})
df

,cusid,payment_method,merchant,total_txn
0,1,DEBIT,SHOPEE,1
1,1,DEBIT,GRAB,2
2,1,CREDIT,SHOPEE,3
3,2,CREDIT,SHOPEE,4
4,2,CREDIT,LAZADA,5
5,2,DEBIT,GSM,6


In [ ]:
pivot_df = df.pivot_table(index=["cusid"], columns=["payment_method", "merchant"], values=["total_txn"])
pivot_df

payment_method CREDIT        DEBIT            
merchant       LAZADA SHOPEE  GRAB  GSM SHOPEE
cusid                                         
1                 NaN    3.0   2.0  NaN    1.0
2                 5.0    4.0   NaN  6.0    NaN

In [ ]:
# MultiIndex
pivot_df.columns

MultiIndex([('CREDIT', 'LAZADA'),
            ('CREDIT', 'SHOPEE'),
            ( 'DEBIT',   'GRAB'),
            ( 'DEBIT',    'GSM'),
            ( 'DEBIT', 'SHOPEE')],
           names=['payment_method', 'merchant'])

In [21]:
flatten_name_df = pivot_df.copy()
flatten_name_df.columns = list(map("_".join, pivot_df.columns))
flatten_name_df

,CREDIT_LAZADA,CREDIT_SHOPEE,DEBIT_GRAB,DEBIT_GSM,DEBIT_SHOPEE
cusid,,,,,
1,NaN,3.0,2.0,NaN,1.0
2,5.0,4.0,NaN,6.0,NaN


## Melt (Unpivot)
- Unpivot a DataFrame from **wide** to **long** format, optionally leaving identifiers set.
- For example, we want to melt the dataframe `df` below into `subjects` and `grades` for each student instead of having multiple subjects columns

In [23]:
df = pd.DataFrame({'Name': ['Bob', 'John', 'Foo', 'Bar', 'Alex', 'Tom'],
                   'Math': ['A+', 'B', 'A', 'F', 'D', 'C'],
                   'English': ['C', 'B', 'B', 'A+', 'F', 'A'],
                   'Age': [13, 16, 16, 15, 15, 13]})
df

,Name,Math,English,Age
0,Bob,A+,C,13
1,John,B,B,16
2,Foo,A,B,16
3,Bar,F,A+,15
4,Alex,D,F,15
5,Tom,C,A,13


In [25]:
df.melt(
    id_vars=["Name", "Age"],
    value_vars=["Math", "English"],
    var_name="Subject",
    value_name="Grades",
).sort_values(by=["Name"])

,Name,Age,Subject,Grades
4,Alex,15,Math,D
10,Alex,15,English,F
3,Bar,15,Math,F
9,Bar,15,English,A+
0,Bob,13,Math,A+
6,Bob,13,English,C
2,Foo,16,Math,A
8,Foo,16,English,B
1,John,16,Math,B
7,John,16,English,B


## Crosstab
- Crosstab: displays the relationship between two or more categorical variables by showing the frequency of different combinations of those variables

In [31]:
df = pd.DataFrame({'Gender': ['Male', 'Female', 'Female', 'Male', 'Male','Female', 'Male'],
        'Education': ['Graduate', 'Undergraduate', 'Undergraduate', 'Graduate', 'Graduate', 'Graduate', 'Undergraduate'],
        'Age': [27, 18, 19, 24, 29, 23,18]})
df

,Gender,Education,Age
0,Male,Graduate,27
1,Female,Undergraduate,18
2,Female,Undergraduate,19
3,Male,Graduate,24
4,Male,Graduate,29
5,Female,Graduate,23
6,Male,Undergraduate,18


In [28]:
# Basic crosstab
cross_tab = pd.crosstab(df['Gender'], df['Education'])
cross_tab

Education,Graduate,Undergraduate
Gender,,
Female,0,2
Male,3,0


In [ ]:
# Crosstab with normalization: shows the proportion of each combination relative to the total.
cross_tab_normalized = pd.crosstab(df['Gender'], df['Education'], normalize='all')
cross_tab_normalized

Education,Graduate,Undergraduate
Gender,,
Female,0.142857,0.285714
Male,0.428571,0.142857


In [ ]:
# Crosstab with aggregation for each combination
cross_tab_agg = pd.crosstab(df['Gender'], df['Education'], values=df['Age'], aggfunc='mean')
cross_tab_agg

Education,Graduate,Undergraduate
Gender,,
Female,23.000000,18.5
Male,26.666667,18.0


In [ ]:


# Crosstab with margins
cross_tab_margins = pd.crosstab(df['Gender'], df['Education'], margins=True, margins_name="Total")
print("\nCrosstab with Margins:")
print(cross_tab_margins)


print("\nCrosstab with Normalization:")
print(cross_tab_normalized)